In [98]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from app.classifier import EnsembleAllNumeric, normalize, get_voting_classifier
from app.main import normalize

df = pd.read_csv("data/processed_data.csv")
df = normalize(df)

val_df = pd.read_csv("data/validation_data_processed.csv")
val_df = normalize(val_df)

_ = df.pop("readme")
_ = val_df.pop("readme")

y = df.pop("label")
y_val = val_df.pop("label")

_ = df.pop("Unnamed: 0")
_ = val_df.pop("Unnamed: 0")

ensemble_clf = EnsembleAllNumeric()
ensemble_clf.fit(df, y)
df = ensemble_clf.transform(df)
useful = ensemble_clf.useful_features

val_df = ensemble_clf.transform(val_df)
val_df = ensemble_clf.keep_useful_features(val_df, useful)

clfs = [clf[1] for clf in get_voting_classifier().estimators]
clfs.append(ensemble_clf)
for clf in clfs:
    clf.fit(df, y)
    print clf.__class__
    print clf.score(val_df, y_val)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
<class 'app.classifier.NaiveBayes'>
0.41935483871
<class 'app.classifier.DecisionTree'>
0.129032258065
<class 'app.classifier.Forest'>
0.41935483871
<class 'app.classifier.KNeighbors'>
0.258064516129
<class 'app.classifier.SVM'>
0.193548387097
<class 'app.classifier.GradBoost'>
0.129032258065
<class 'app.classifier.XGBoost'>


ValueError: feature_names mismatch: ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', '

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] criterion=gini, max_depth=6 .....................................
[CV] criterion=gini, max_depth=6 .....................................
[CV] criterion=gini, max_depth=6 .....................................
[CV] criterion=gini, max_depth=4 .....................................
[CV] criterion=gini, max_depth=4 .....................................
[CV] criterion=gini, max_depth=4 .....................................
[CV] criterion=gini, max_depth=12 ....................................
[CV] criterion=gini, max_depth=12 ....................................
[CV] ............................ criterion=gini, max_depth=4 -   0.1s
[CV] criterion=gini, max_depth=12 ....................................
[CV] ............................ criterion=gini, max_depth=4 -   0.1s
[CV] criterion=entropy, max_depth=7 ..................................
[CV] ........................... criterion=gini, max_depth=12 -   0.1s
[CV] criterion=e

[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   26.0s finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=<app.classifier.EnsembleAllNumeric object at 0x7f42305cad50>,
          fit_params={}, iid=True, n_iter=10, n_jobs=-1,
          param_distributions={'criterion': ['gini', 'entropy'], 'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f42305ca210>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [80]:
val_df = clf.transform(val_df)
val_df = clf.keep_useful_features(val_df, clf.useful_features)
random_search.score(val_df, y_val)

0.35483870967741937

In [81]:
random_search.best_score_

0.31515580736543908

In [82]:
random_search.best_params_

{'criterion': 'gini', 'max_depth': 15}

In [83]:
df.head()

,watchers,mentionableUsers,closed_pull_requests,closed_issues,open_issues,forks,merged_pull_requests,stargazers,open_pull_requests,projects,...,description_written,description_www,description_xcode,description_year,description_yields,description_yorgey,description_you,description_your,description_yourself,description_youtube
0,20,1,2,-2,2,111,0,137,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3,188,17,-237,3,137,229,19,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9,130,30,-241,14,156,274,104,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,14,2,0,0,0,55,1,267,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,75,29,8,-56,16,145,70,296,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
from sklearn.tree import export_graphviz
export_graphviz(clf.clf, out_file='tree.dot', class_names=clf.clf.classes_, feature_names=clf.useful_features) 

AttributeError: 'RandomForestClassifier' object has no attribute 'tree_'

In [ ]:
!dot -Tpng tree.dot -o tree.png 

In [ ]:
!eog tree.png

In [18]:
df.loc[df["branchesCount"] == 0]

,repository,owner,name,watchers,mentionableUsers,closed_pull_requests,closed_issues,open_issues,forks,merged_pull_requests,...,LANGUAGE_ANTLR,LANGUAGE_Scilab,LANGUAGE_Module Management System,LANGUAGE_SAS,LANGUAGE_Nemerle,LANGUAGE_Csound Document,LANGUAGE_Agda,LANGUAGE_XQuery,LANGUAGE_Ada,LANGUAGE_Arduino
69,tekkub/tekkub.github.com,tekkub,tekkub.github.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,hacke2/hacke2.github.,hacke2,hacke2.github.,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
327,cdwanze/cdwanze.github.io,cdwanze,cdwanze.github.io,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
486,datasets/datasets.github.com,datasets,datasets.github.com,6.0,23.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
593,meebert/Dynamic-Programming-EggDrop,meebert,Dynamic-Programming-EggDrop,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
613,myhomeworkhelp/Accounts-Assignment-Help,myhomeworkhelp,Accounts-Assignment-Help,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
634,theironyard-rails-atl/jamesdab,theironyard-rails-atl,jamesdab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1019,XuezhengMa/Math,XuezhengMa,Math,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1043,studentalpha/PRML-solutions,studentalpha,PRML-solutions,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1044,bachkiko/SoftUni,bachkiko,SoftUni,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
import numpy as np

from time import time
from scipy.stats import randint as sp_randint
from operator import itemgetter
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier
from app.classifier import EnsembleAllNumeric, normalize

clf = EnsembleAllNumeric()

param_dist = {"max_depth": sp_randint(3, 20),
              "criterion": ["gini", "entropy"]}

n_iter_search = 10
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search, verbose=2, n_jobs=-1)
df = clf.transform(df)
clf.useful_features = list(df.columns)
random_search.fit(df, y)

0.20821529745042494